<a href="https://colab.research.google.com/github/elangbijak4/Quantum_Algorithm_Almudaya/blob/main/Rev1_Quantum_PoW_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq

In [3]:
import hashlib
import datetime as date
import cirq

In [4]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash, nonce=0):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        sha = hashlib.sha256()
        sha.update(str(self.index).encode('utf-8') +
                   str(self.timestamp).encode('utf-8') +
                   str(self.data).encode('utf-8') +
                   str(self.previous_hash).encode('utf-8') +
                   str(self.nonce).encode('utf-8'))
        return sha.hexdigest()

In [5]:
class QuantumProofOfWork:
    def __init__(self, difficulty):
        self.difficulty = difficulty

    def run_quantum_circuit(self, nonce):
        # Membuat qubit untuk nonce
        qubits = cirq.LineQubit.range(3)

        # Membuat sirkuit kuantum sederhana
        circuit = cirq.Circuit(
            cirq.H(qubits[0]),
            cirq.CX(qubits[0], qubits[1]),
            cirq.X(qubits[1]) ** nonce,
            cirq.H(qubits[1]),
            cirq.measure(qubits[1], key='result')
        )

        # Simulasi sirkuit kuantum
        simulator = cirq.Simulator()
        result = simulator.run(circuit, repetitions=1)

        # Mendapatkan hasil dari pengukuran
        measured_result = result.measurements['result'][0][0]  # Ambil elemen pertama dari array hasil
        return int(measured_result)

    def mine(self, block):
        nonce = 0
        target = 2 ** (256 - self.difficulty)
        while True:
            block.nonce = nonce
            hash_result = int(block.calculate_hash(), 16)
            quantum_hash = self.run_quantum_circuit(nonce)
            if hash_result < target and quantum_hash == 0:  # hash_result meets difficulty and quantum_hash condition
                return nonce, quantum_hash
            else:
                nonce += 1

In [6]:
class Blockchain:
    def __init__(self, difficulty):
        self.chain = [self.create_genesis_block()]
        self.difficulty = difficulty
        self.qpow = QuantumProofOfWork(self.difficulty)

    def create_genesis_block(self):
        return Block(0, date.datetime.now(), "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        nonce, quantum_hash = self.qpow.mine(new_block)
        new_block.nonce = nonce
        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)
        print(f"Block mined with Nonce: {nonce} and Quantum Hash: {quantum_hash}")

In [7]:
# Membuat blockchain dengan kesulitan tertentu
difficulty = 4
my_blockchain = Blockchain(difficulty)

In [8]:
# Menambahkan beberapa blok
my_blockchain.add_block(Block(1, date.datetime.now(), {'amount': 4}, ''))
my_blockchain.add_block(Block(2, date.datetime.now(), {'amount': 10}, ''))

Block mined with Nonce: 11 and Quantum Hash: 0
Block mined with Nonce: 87 and Quantum Hash: 0


In [9]:
# Menampilkan blockchain
for block in my_blockchain.chain:
    print(f"Index: {block.index}")
    print(f"Timestamp: {block.timestamp}")
    print(f"Data: {block.data}")
    print(f"Previous Hash: {block.previous_hash}")
    print(f"Hash: {block.hash}")
    print(f"Nonce: {block.nonce}")
    print("\n")

Index: 0
Timestamp: 2024-06-18 16:56:58.034978
Data: Genesis Block
Previous Hash: 0
Hash: afcee504b431b4da013a101d49f214f4a24ed13df04c6adf8390f8589c01db0e
Nonce: 0


Index: 1
Timestamp: 2024-06-18 16:57:01.213003
Data: {'amount': 4}
Previous Hash: afcee504b431b4da013a101d49f214f4a24ed13df04c6adf8390f8589c01db0e
Hash: 0853dc1435df1625c3f237a2398f742fd9b56bbce33ddb2fc99571cca6311125
Nonce: 11


Index: 2
Timestamp: 2024-06-18 16:57:01.241857
Data: {'amount': 10}
Previous Hash: 0853dc1435df1625c3f237a2398f742fd9b56bbce33ddb2fc99571cca6311125
Hash: 08428a9ac093f0d88c25c6dca7d07167ad2664b520332af387389b1565a95bf7
Nonce: 87


